### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]  # number of rows in the matrix
    n_movies = ratings_mat.shape[1]  # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))  # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features)  # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies)  # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])
        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10) 


Optimization Statistics
Iterations | Mean Squared Error 
1 		 41.656693
2 		 13.529621
3 		 3.785011
4 		 2.823646
5 		 2.743808
6 		 2.706190
7 		 2.672204
8 		 2.637015
9 		 2.598928
10 		 2.556942


In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.12782443   8.86400158  10.6573912   10.09019778]
 [  8.83871948   7.59442982   8.96189126   8.67920667]
 [  7.80013254   6.89744033   8.61033796   7.5214499 ]
 [  9.59273078   8.37769782  10.26958489   9.2994272 ]
 [  8.87784361   7.49415119   8.98241611   8.27841674]
 [  6.6539904    6.03290234   7.43549234   6.77290263]
 [  9.18059683   8.09057007   9.74162247   9.11471834]
 [  8.45628994   7.14693555   8.42816258   8.21750141]
 [  8.55668285   7.4114006    8.68835522   8.63412091]
 [  7.69001571   6.72368934   7.88129783   7.81876797]
 [  8.78628381   7.79943707   9.54975437   8.68965841]
 [  9.84332573   8.3828817    9.97696414   9.48497857]
 [  9.39207035   8.09338344   9.48213459   9.39161771]
 [  6.40257287   5.80101884   7.06309621   6.63596936]
 [  9.60154547   8.50814254  10.14551913   9.89817835]
 [  9.54678056   8.51810884  10.15903377   9.91464119]
 [  8.44417695   7.23749365   8.5005138    8.38112129]
 [  7.27763735   6.15617971   6.87827223   7.4406938 ]
 [  8.2743

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 41.504820
2 		 13.966639
3 		 4.047253
4 		 2.813082
5 		 2.676561
6 		 2.631451
7 		 2.595729
8 		 2.557925
9 		 2.515379
10 		 2.466977
11 		 2.411894
12 		 2.349415
13 		 2.278962
14 		 2.200170
15 		 2.112979
16 		 2.017717
17 		 1.915173
18 		 1.806634
19 		 1.693857
20 		 1.578984
21 		 1.464385
22 		 1.352445
23 		 1.245355
24 		 1.144916
25 		 1.052423
26 		 0.968621
27 		 0.893750
28 		 0.827631
29 		 0.769781
30 		 0.719530
31 		 0.676107
32 		 0.638707
33 		 0.606539
34 		 0.578846
35 		 0.554925
36 		 0.534131
37 		 0.515884
38 		 0.499668
39 		 0.485030
40 		 0.471576
41 		 0.458971
42 		 0.446927
43 		 0.435206
44 		 0.423609
45 		 0.411973
46 		 0.400170
47 		 0.388096
48 		 0.375679
49 		 0.362865
50 		 0.349627
51 		 0.335954
52 		 0.321858
53 		 0.307370
54 		 0.292534
55 		 0.277413
56 		 0.262085
57 		 0.246637
58 		 0.231167
59 		 0.215778
60 		 0.200576
61 		 0.185668
62 		 0.171154
63 		 0.157130
64 		

In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00000067  10.00000013  10.00000009   9.99999946]
 [ 10.00002703   4.00000893   9.00000696   9.99997491]
 [  8.00001113   9.00000337  10.00000254   4.99998976]
 [  9.00000925   8.00000257  10.00000183   9.9999913 ]
 [  9.99997876   4.99999253   8.99999398   9.00001938]
 [  5.99998483   3.9999948    9.99999588   6.00001397]
 [  9.00001647   8.00000524  10.00000403   8.99998477]
 [ 10.00000563   5.00000162   9.00000117   7.99999464]
 [  6.99997069   7.99999      9.9999921    8.00002708]
 [  8.9999672    4.99998923   8.99999161   7.00003031]
 [  8.99998323   7.99999478   9.99999606   8.0000157 ]
 [  8.99999761   9.99999962   9.99999989   9.00000252]
 [ 10.00000375   9.00000169  10.00000152   7.99999683]
 [  4.99998726   7.99999591   4.99999689   8.00001197]
 [ 10.00001416   8.00000515  10.00000422   9.99998715]
 [  8.99999588   8.99999892   9.99999928  10.00000402]
 [  9.00002163   8.00000734   8.00000582   7.99998016]
 [  9.9999905    7.9999967    0.99999737  10.00000871]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [8]:
# run SVD on the matrix with the missing value
#use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 39.041382
2 		 12.829691
3 		 3.983584
4 		 2.785833
5 		 2.589553
6 		 2.516091
7 		 2.465612
8 		 2.418294
9 		 2.369281
10 		 2.317406
11 		 2.262545
12 		 2.204949
13 		 2.145035
14 		 2.083300
15 		 2.020255
16 		 1.956372
17 		 1.892034
18 		 1.827497
19 		 1.762877
20 		 1.698156
21 		 1.633212
22 		 1.567867
23 		 1.501944
24 		 1.435332
25 		 1.368034
26 		 1.300205
27 		 1.232169
28 		 1.164411
29 		 1.097538
30 		 1.032234
31 		 0.969192
32 		 0.909044
33 		 0.852307
34 		 0.799339
35 		 0.750315
36 		 0.705230
37 		 0.663919
38 		 0.626091
39 		 0.591367
40 		 0.559321
41 		 0.529514
42 		 0.501525
43 		 0.474968
44 		 0.449511
45 		 0.424876
46 		 0.400848
47 		 0.377271
48 		 0.354046
49 		 0.331126
50 		 0.308510
51 		 0.286236
52 		 0.264376
53 		 0.243023
54 		 0.222289
55 		 0.202291
56 		 0.183146
57 		 0.164961
58 		 0.147832
59 		 0.131833
60 		 0.117015
61 		 0.103405
62 		 0.091006
63 		 0.079798
64 		

In [9]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 8.473436075241562:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 
1 		 23.503286
2 		 10.772922
3 		 7.401385
4 		 5.712513
5 		 4.648335
6 		 3.905292
7 		 3.353208
8 		 2.925448
9 		 2.583875
10 		 2.304902
11 		 2.073157
12 		 1.878234
13 		 1.712833
14 		 1.571617
15 		 1.450500
16 		 1.346230
17 		 1.256133
18 		 1.177980
19 		 1.109896
20 		 1.050305


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [11]:
# How many actual ratings exist in first_1000_users
num_ratings =np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [12]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
